In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2 
import PIL 
import gc
import matplotlib.pyplot as plt
from sklearn.model_selection import  train_test_split
from tensorflow import set_random_seed
from tqdm import tqdm
from math import ceil

import keras
from keras.models import Sequential, Model
from keras.preprocessing import image  
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Input
from keras.layers import Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.activations import softmax
from keras.optimizers import Adam, rmsprop
from tqdm import tqdm

import psutil
import seaborn as sns
from keras.utils.np_utils import to_categorical


from keras.applications.densenet import DenseNet121

from keras.layers import BatchNormalization

Using TensorFlow backend.


In [2]:
df_train=pd.read_csv("train.csv") 
df_test=pd.read_csv("test.csv")

In [3]:
df_train['id_code'] = 'train_images/' + df_train['id_code'].astype(str) + '.png'
df_test['id_code'] = 'test_images/' + df_test['id_code'].astype(str) + '.png'

In [4]:
SEED=42
np.random.seed(SEED) 
set_random_seed(SEED)

In [5]:
SEED=42
IMG_DIM=224 #512
BATCH_SIZE = 16
CHANNEL_SIZE=3
N_EPOCHS=10
N_CLASSES = df_train['diagnosis'].nunique()
CLASSS={0:"No DR",1:"Mild",2:"Moderate",3:"Severe",4:"Proliferative DR"}

In [6]:
def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

In [7]:
def preprocess_image(path,sigmaX = 10): # default value is taken as 10, unless specified
    image = cv2.cvtColor(cv2.imread(path),cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image,(IMG_DIM,IMG_DIM))
    image = cv2.addWeighted(image,4,cv2.GaussianBlur(image,(0,0),IMG_DIM/30),-4,128)
    
    return image

In [8]:
# from tqdm import tqdm_notebook


# N = df_train.shape[0]
# x_train = np.empty((N, IMG_DIM, IMG_DIM, 3), dtype=np.uint8)

# for i, image_id in enumerate(tqdm_notebook(df_train['id_code'])):
#     path = f"{image_id}"
#     x_train[i, :, :, :] = preprocess_image(path)

KeyboardInterrupt: 

In [10]:
# np.save('processed_train_densenet', x_train)

In [8]:
x_train = np.load('processed_train_images.npy')
x_train.shape

(3662, 224, 224, 3)

In [11]:
# N = df_test.shape[0]
# test = np.empty((N, IMG_DIM, IMG_DIM, 3), dtype=np.uint8)

# for i, image_id in enumerate(tqdm_notebook(df_test['id_code'])):
#     path = f"{image_id}"
#     test[i, :, :, :] = preprocess_image(path)

In [12]:
# np.save('processed_test_densenet', test)

In [9]:
test = np.load('processed_test_images.npy')
test.shape

(1928, 224, 224, 3)

In [10]:
y_train = to_categorical(df_train['diagnosis'])

In [11]:
X_train,X_test,Y_train,Y_test = train_test_split(x_train,y_train,test_size=0.10,random_state=42,stratify=y_train)

In [12]:
def create_densenet(img_dim,CHANNEL,n_class):
    input_tensor=Input(shape=(img_dim, img_dim,CHANNEL))
    base_model = DenseNet121(weights=None,
                          include_top=False,
                          input_tensor=input_tensor)
    x=GlobalAveragePooling2D()(base_model.output)
    x=Dropout(0.3)(x)
    x=Dense(1024, activation='relu')(x)
    x=Dense(512, activation='relu')(x)
    x=Dropout(0.3)(x)
    x=BatchNormalization()(x)
    output_layer=Dense(n_class,activation='softmax', name="Output_Layer")(x)
    model_densenet =Model(input_tensor, output_layer)
    return model_densenet
model=create_densenet(224,3, 5)
model.summary()
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001,decay=1e-6), metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormal

                                                                 conv3_block3_2_conv[0][0]        
__________________________________________________________________________________________________
conv3_block4_0_bn (BatchNormali (None, 28, 28, 224)  896         conv3_block3_concat[0][0]        
__________________________________________________________________________________________________
conv3_block4_0_relu (Activation (None, 28, 28, 224)  0           conv3_block4_0_bn[0][0]          
__________________________________________________________________________________________________
conv3_block4_1_conv (Conv2D)    (None, 28, 28, 128)  28672       conv3_block4_0_relu[0][0]        
__________________________________________________________________________________________________
conv3_block4_1_bn (BatchNormali (None, 28, 28, 128)  512         conv3_block4_1_conv[0][0]        
__________________________________________________________________________________________________
conv3_bloc

conv4_block2_1_conv (Conv2D)    (None, 14, 14, 128)  36864       conv4_block2_0_relu[0][0]        
__________________________________________________________________________________________________
conv4_block2_1_bn (BatchNormali (None, 14, 14, 128)  512         conv4_block2_1_conv[0][0]        
__________________________________________________________________________________________________
conv4_block2_1_relu (Activation (None, 14, 14, 128)  0           conv4_block2_1_bn[0][0]          
__________________________________________________________________________________________________
conv4_block2_2_conv (Conv2D)    (None, 14, 14, 32)   36864       conv4_block2_1_relu[0][0]        
__________________________________________________________________________________________________
conv4_block2_concat (Concatenat (None, 14, 14, 320)  0           conv4_block1_concat[0][0]        
                                                                 conv4_block2_2_conv[0][0]        
__________

__________________________________________________________________________________________________
conv4_block13_1_conv (Conv2D)   (None, 14, 14, 128)  81920       conv4_block13_0_relu[0][0]       
__________________________________________________________________________________________________
conv4_block13_1_bn (BatchNormal (None, 14, 14, 128)  512         conv4_block13_1_conv[0][0]       
__________________________________________________________________________________________________
conv4_block13_1_relu (Activatio (None, 14, 14, 128)  0           conv4_block13_1_bn[0][0]         
__________________________________________________________________________________________________
conv4_block13_2_conv (Conv2D)   (None, 14, 14, 32)   36864       conv4_block13_1_relu[0][0]       
__________________________________________________________________________________________________
conv4_block13_concat (Concatena (None, 14, 14, 672)  0           conv4_block12_concat[0][0]       
          

conv4_block24_0_relu (Activatio (None, 14, 14, 992)  0           conv4_block24_0_bn[0][0]         
__________________________________________________________________________________________________
conv4_block24_1_conv (Conv2D)   (None, 14, 14, 128)  126976      conv4_block24_0_relu[0][0]       
__________________________________________________________________________________________________
conv4_block24_1_bn (BatchNormal (None, 14, 14, 128)  512         conv4_block24_1_conv[0][0]       
__________________________________________________________________________________________________
conv4_block24_1_relu (Activatio (None, 14, 14, 128)  0           conv4_block24_1_bn[0][0]         
__________________________________________________________________________________________________
conv4_block24_2_conv (Conv2D)   (None, 14, 14, 32)   36864       conv4_block24_1_relu[0][0]       
__________________________________________________________________________________________________
conv4_bloc

conv5_block10_1_relu (Activatio (None, 7, 7, 128)    0           conv5_block10_1_bn[0][0]         
__________________________________________________________________________________________________
conv5_block10_2_conv (Conv2D)   (None, 7, 7, 32)     36864       conv5_block10_1_relu[0][0]       
__________________________________________________________________________________________________
conv5_block10_concat (Concatena (None, 7, 7, 832)    0           conv5_block9_concat[0][0]        
                                                                 conv5_block10_2_conv[0][0]       
__________________________________________________________________________________________________
conv5_block11_0_bn (BatchNormal (None, 7, 7, 832)    3328        conv5_block10_concat[0][0]       
__________________________________________________________________________________________________
conv5_block11_0_relu (Activatio (None, 7, 7, 832)    0           conv5_block11_0_bn[0][0]         
__________

In [24]:
model.fit(X_train, Y_train, batch_size=16, epochs=20,validation_data=(X_test,Y_test), verbose=1)

Train on 3295 samples, validate on 367 samples
Epoch 1/20
3295/3295 [==============================] - ETA: 32:05 - loss: 0.0015 - acc: 1.00 - ETA: 16:22 - loss: 0.0012 - acc: 1.00 - ETA: 11:08 - loss: 9.7842e-04 - acc: 1.00 - ETA: 8:31 - loss: 0.0011 - acc: 1.0000     - ETA: 6:56 - loss: 0.0126 - acc: 0.987 - ETA: 5:53 - loss: 0.0106 - acc: 0.989 - ETA: 5:08 - loss: 0.0091 - acc: 0.991 - ETA: 4:34 - loss: 0.0088 - acc: 0.992 - ETA: 4:08 - loss: 0.0110 - acc: 0.993 - ETA: 3:46 - loss: 0.0099 - acc: 0.993 - ETA: 3:29 - loss: 0.0131 - acc: 0.988 - ETA: 3:15 - loss: 0.0129 - acc: 0.989 - ETA: 3:02 - loss: 0.0122 - acc: 0.990 - ETA: 2:52 - loss: 0.0151 - acc: 0.986 - ETA: 2:42 - loss: 0.0141 - acc: 0.987 - ETA: 2:34 - loss: 0.0132 - acc: 0.988 - ETA: 2:27 - loss: 0.0125 - acc: 0.989 - ETA: 2:21 - loss: 0.0122 - acc: 0.989 - ETA: 2:15 - loss: 0.0143 - acc: 0.986 - ETA: 2:10 - loss: 0.0137 - acc: 0.987 - ETA: 2:05 - loss: 0.0131 - acc: 0.988 - ETA: 2:01 - loss: 0.0125 - acc: 0.988 - ETA: 1:5

3295/3295 [==============================] - ETA: 4:23 - loss: 0.0081 - acc: 1.000 - ETA: 4:22 - loss: 0.0114 - acc: 1.000 - ETA: 4:21 - loss: 0.0163 - acc: 1.000 - ETA: 4:20 - loss: 0.0175 - acc: 1.000 - ETA: 4:19 - loss: 0.0331 - acc: 0.987 - ETA: 4:18 - loss: 0.0277 - acc: 0.989 - ETA: 4:17 - loss: 0.0307 - acc: 0.991 - ETA: 4:16 - loss: 0.0351 - acc: 0.984 - ETA: 4:14 - loss: 0.0491 - acc: 0.979 - ETA: 4:13 - loss: 0.0451 - acc: 0.981 - ETA: 4:11 - loss: 0.0422 - acc: 0.983 - ETA: 4:10 - loss: 0.0397 - acc: 0.984 - ETA: 4:09 - loss: 0.0422 - acc: 0.980 - ETA: 4:08 - loss: 0.0397 - acc: 0.982 - ETA: 4:06 - loss: 0.0371 - acc: 0.983 - ETA: 4:05 - loss: 0.0352 - acc: 0.984 - ETA: 4:04 - loss: 0.0333 - acc: 0.985 - ETA: 4:02 - loss: 0.0317 - acc: 0.986 - ETA: 4:01 - loss: 0.0349 - acc: 0.983 - ETA: 4:00 - loss: 0.0332 - acc: 0.984 - ETA: 3:59 - loss: 0.0318 - acc: 0.985 - ETA: 3:58 - loss: 0.0317 - acc: 0.985 - ETA: 3:56 - loss: 0.0306 - acc: 0.986 - ETA: 3:55 - loss: 0.0296 - acc: 0.9

Epoch 4/20
3295/3295 [==============================] - ETA: 4:22 - loss: 5.0775e-04 - acc: 1.000 - ETA: 4:21 - loss: 0.0182 - acc: 1.0000    - ETA: 4:21 - loss: 0.0181 - acc: 1.000 - ETA: 4:22 - loss: 0.0174 - acc: 1.000 - ETA: 4:22 - loss: 0.0229 - acc: 1.000 - ETA: 4:20 - loss: 0.0269 - acc: 1.000 - ETA: 4:18 - loss: 0.0232 - acc: 1.000 - ETA: 4:17 - loss: 0.0204 - acc: 1.000 - ETA: 4:16 - loss: 0.0268 - acc: 1.000 - ETA: 4:14 - loss: 0.0335 - acc: 0.993 - ETA: 4:13 - loss: 0.0315 - acc: 0.994 - ETA: 4:12 - loss: 0.0292 - acc: 0.994 - ETA: 4:10 - loss: 0.0283 - acc: 0.995 - ETA: 4:09 - loss: 0.0359 - acc: 0.986 - ETA: 4:08 - loss: 0.0357 - acc: 0.987 - ETA: 4:07 - loss: 0.0386 - acc: 0.984 - ETA: 4:06 - loss: 0.0364 - acc: 0.985 - ETA: 4:05 - loss: 0.0344 - acc: 0.986 - ETA: 4:03 - loss: 0.0349 - acc: 0.986 - ETA: 4:02 - loss: 0.0364 - acc: 0.987 - ETA: 4:00 - loss: 0.0349 - acc: 0.988 - ETA: 3:59 - loss: 0.0352 - acc: 0.988 - ETA: 3:58 - loss: 0.0337 - acc: 0.989 - ETA: 3:56 - loss

Epoch 5/20
3295/3295 [==============================] - ETA: 4:23 - loss: 4.3428e-04 - acc: 1.000 - ETA: 4:21 - loss: 5.2316e-04 - acc: 1.000 - ETA: 4:20 - loss: 4.6106e-04 - acc: 1.000 - ETA: 4:20 - loss: 6.0136e-04 - acc: 1.000 - ETA: 4:19 - loss: 8.1458e-04 - acc: 1.000 - ETA: 4:18 - loss: 0.0055 - acc: 1.0000    - ETA: 4:16 - loss: 0.0328 - acc: 0.991 - ETA: 4:15 - loss: 0.0290 - acc: 0.992 - ETA: 4:14 - loss: 0.0260 - acc: 0.993 - ETA: 4:12 - loss: 0.0243 - acc: 0.993 - ETA: 4:11 - loss: 0.0240 - acc: 0.994 - ETA: 4:10 - loss: 0.0226 - acc: 0.994 - ETA: 4:09 - loss: 0.0217 - acc: 0.995 - ETA: 4:08 - loss: 0.0208 - acc: 0.995 - ETA: 4:07 - loss: 0.0201 - acc: 0.995 - ETA: 4:05 - loss: 0.0241 - acc: 0.992 - ETA: 4:04 - loss: 0.0230 - acc: 0.992 - ETA: 4:03 - loss: 0.0220 - acc: 0.993 - ETA: 4:02 - loss: 0.0212 - acc: 0.993 - ETA: 4:01 - loss: 0.0207 - acc: 0.993 - ETA: 3:59 - loss: 0.0205 - acc: 0.994 - ETA: 3:58 - loss: 0.0196 - acc: 0.994 - ETA: 3:57 - loss: 0.0190 - acc: 0.994 - 

Epoch 6/20
3295/3295 [==============================] - ETA: 48s - loss: 0.2698 - acc: 0.81 - ETA: 48s - loss: 0.1384 - acc: 0.90 - ETA: 48s - loss: 0.1120 - acc: 0.91 - ETA: 47s - loss: 0.0873 - acc: 0.93 - ETA: 47s - loss: 0.0795 - acc: 0.93 - ETA: 47s - loss: 0.0672 - acc: 0.94 - ETA: 47s - loss: 0.0579 - acc: 0.95 - ETA: 46s - loss: 0.0520 - acc: 0.96 - ETA: 46s - loss: 0.0464 - acc: 0.96 - ETA: 46s - loss: 0.0423 - acc: 0.96 - ETA: 46s - loss: 0.0419 - acc: 0.97 - ETA: 45s - loss: 0.0388 - acc: 0.97 - ETA: 45s - loss: 0.0359 - acc: 0.97 - ETA: 45s - loss: 0.0335 - acc: 0.97 - ETA: 45s - loss: 0.0388 - acc: 0.97 - ETA: 44s - loss: 0.0381 - acc: 0.97 - ETA: 44s - loss: 0.0360 - acc: 0.97 - ETA: 44s - loss: 0.0340 - acc: 0.97 - ETA: 44s - loss: 0.0369 - acc: 0.97 - ETA: 43s - loss: 0.0359 - acc: 0.97 - ETA: 43s - loss: 0.0342 - acc: 0.97 - ETA: 43s - loss: 0.0402 - acc: 0.97 - ETA: 43s - loss: 0.0388 - acc: 0.97 - ETA: 43s - loss: 0.0373 - acc: 0.97 - ETA: 42s - loss: 0.0373 - acc: 0

...

3295/3295 [==============================] - ETA: 48s - loss: 2.9506e-04 - acc: 1.00 - ETA: 50s - loss: 0.0011 - acc: 1.0000   - ETA: 50s - loss: 0.0289 - acc: 0.97 - ETA: 50s - loss: 0.0217 - acc: 0.98 - ETA: 50s - loss: 0.0256 - acc: 0.98 - ETA: 49s - loss: 0.0307 - acc: 0.97 - ETA: 48s - loss: 0.0277 - acc: 0.98 - ETA: 48s - loss: 0.0259 - acc: 0.98 - ETA: 48s - loss: 0.0297 - acc: 0.97 - ETA: 47s - loss: 0.0269 - acc: 0.98 - ETA: 47s - loss: 0.0291 - acc: 0.97 - ETA: 47s - loss: 0.0267 - acc: 0.97 - ETA: 47s - loss: 0.0248 - acc: 0.98 - ETA: 46s - loss: 0.0242 - acc: 0.98 - ETA: 46s - loss: 0.0227 - acc: 0.98 - ETA: 46s - loss: 0.0213 - acc: 0.98 - ETA: 45s - loss: 0.0201 - acc: 0.98 - ETA: 45s - loss: 0.0190 - acc: 0.98 - ETA: 45s - loss: 0.0184 - acc: 0.98 - ETA: 45s - loss: 0.0175 - acc: 0.98 - ETA: 44s - loss: 0.0167 - acc: 0.98 - ETA: 44s - loss: 0.0160 - acc: 0.98 - ETA: 44s - loss: 0.0154 - acc: 0.98 - ETA: 44s - loss: 0.0190 - acc: 0.98 - ETA: 43s - loss: 0.0183 - acc: 0.98

3295/3295 [==============================] - ETA: 4:25 - loss: 9.7806e-04 - acc: 1.000 - ETA: 4:23 - loss: 7.8221e-04 - acc: 1.000 - ETA: 4:22 - loss: 0.0270 - acc: 0.9792    - ETA: 4:20 - loss: 0.0205 - acc: 0.984 - ETA: 4:19 - loss: 0.0169 - acc: 0.987 - ETA: 4:17 - loss: 0.0141 - acc: 0.989 - ETA: 4:16 - loss: 0.0212 - acc: 0.991 - ETA: 4:14 - loss: 0.0244 - acc: 0.992 - ETA: 4:13 - loss: 0.0217 - acc: 0.993 - ETA: 4:12 - loss: 0.0198 - acc: 0.993 - ETA: 4:10 - loss: 0.0181 - acc: 0.994 - ETA: 4:09 - loss: 0.0166 - acc: 0.994 - ETA: 4:08 - loss: 0.0210 - acc: 0.990 - ETA: 4:06 - loss: 0.0195 - acc: 0.991 - ETA: 4:05 - loss: 0.0182 - acc: 0.991 - ETA: 4:04 - loss: 0.0171 - acc: 0.992 - ETA: 4:02 - loss: 0.0176 - acc: 0.992 - ETA: 4:01 - loss: 0.0199 - acc: 0.989 - ETA: 4:00 - loss: 0.0189 - acc: 0.990 - ETA: 3:59 - loss: 0.0180 - acc: 0.990 - ETA: 3:58 - loss: 0.0202 - acc: 0.991 - ETA: 3:56 - loss: 0.0193 - acc: 0.991 - ETA: 3:55 - loss: 0.0206 - acc: 0.989 - ETA: 3:54 - loss: 0.019

Epoch 15/20
3295/3295 [==============================] - ETA: 48s - loss: 0.1199 - acc: 0.87 - ETA: 48s - loss: 0.0744 - acc: 0.93 - ETA: 48s - loss: 0.0497 - acc: 0.95 - ETA: 48s - loss: 0.0374 - acc: 0.96 - ETA: 47s - loss: 0.0299 - acc: 0.97 - ETA: 47s - loss: 0.0336 - acc: 0.96 - ETA: 47s - loss: 0.0288 - acc: 0.97 - ETA: 47s - loss: 0.0253 - acc: 0.97 - ETA: 47s - loss: 0.0348 - acc: 0.97 - ETA: 47s - loss: 0.0313 - acc: 0.97 - ETA: 46s - loss: 0.0285 - acc: 0.97 - ETA: 46s - loss: 0.0262 - acc: 0.97 - ETA: 46s - loss: 0.0243 - acc: 0.98 - ETA: 45s - loss: 0.0254 - acc: 0.98 - ETA: 45s - loss: 0.0237 - acc: 0.98 - ETA: 45s - loss: 0.0235 - acc: 0.98 - ETA: 45s - loss: 0.0271 - acc: 0.98 - ETA: 45s - loss: 0.0256 - acc: 0.98 - ETA: 44s - loss: 0.0305 - acc: 0.98 - ETA: 44s - loss: 0.0290 - acc: 0.98 - ETA: 44s - loss: 0.0276 - acc: 0.98 - ETA: 44s - loss: 0.0264 - acc: 0.98 - ETA: 43s - loss: 0.0253 - acc: 0.98 - ETA: 43s - loss: 0.0278 - acc: 0.98 - ETA: 43s - loss: 0.0267 - acc: 

3295/3295 [==============================] - ETA: 49s - loss: 0.0016 - acc: 1.00 - ETA: 48s - loss: 0.0010 - acc: 1.00 - ETA: 48s - loss: 0.0022 - acc: 1.00 - ETA: 49s - loss: 0.0516 - acc: 0.98 - ETA: 48s - loss: 0.0424 - acc: 0.98 - ETA: 48s - loss: 0.0361 - acc: 0.98 - ETA: 48s - loss: 0.0313 - acc: 0.99 - ETA: 48s - loss: 0.0339 - acc: 0.99 - ETA: 48s - loss: 0.0302 - acc: 0.99 - ETA: 47s - loss: 0.0272 - acc: 0.99 - ETA: 47s - loss: 0.0248 - acc: 0.99 - ETA: 47s - loss: 0.0229 - acc: 0.99 - ETA: 47s - loss: 0.0212 - acc: 0.99 - ETA: 46s - loss: 0.0198 - acc: 0.99 - ETA: 46s - loss: 0.0234 - acc: 0.99 - ETA: 46s - loss: 0.0220 - acc: 0.99 - ETA: 46s - loss: 0.0207 - acc: 0.99 - ETA: 46s - loss: 0.0284 - acc: 0.98 - ETA: 45s - loss: 0.0270 - acc: 0.99 - ETA: 45s - loss: 0.0257 - acc: 0.99 - ETA: 45s - loss: 0.0247 - acc: 0.99 - ETA: 45s - loss: 0.0237 - acc: 0.99 - ETA: 44s - loss: 0.0228 - acc: 0.99 - ETA: 44s - loss: 0.0244 - acc: 0.98 - ETA: 44s - loss: 0.0278 - acc: 0.99 - ETA: 

3295/3295 [==============================] - ETA: 48s - loss: 0.0257 - acc: 1.00 - ETA: 48s - loss: 0.0170 - acc: 1.00 - ETA: 48s - loss: 0.0118 - acc: 1.00 - ETA: 51s - loss: 0.0095 - acc: 1.00 - ETA: 50s - loss: 0.0078 - acc: 1.00 - ETA: 49s - loss: 0.0081 - acc: 1.00 - ETA: 49s - loss: 0.0072 - acc: 1.00 - ETA: 49s - loss: 0.0068 - acc: 1.00 - ETA: 49s - loss: 0.0069 - acc: 1.00 - ETA: 48s - loss: 0.0084 - acc: 1.00 - ETA: 48s - loss: 0.0085 - acc: 1.00 - ETA: 48s - loss: 0.0151 - acc: 0.99 - ETA: 47s - loss: 0.0140 - acc: 0.99 - ETA: 47s - loss: 0.0173 - acc: 0.99 - ETA: 47s - loss: 0.0186 - acc: 0.99 - ETA: 46s - loss: 0.0174 - acc: 0.99 - ETA: 46s - loss: 0.0167 - acc: 0.99 - ETA: 46s - loss: 0.0167 - acc: 0.99 - ETA: 46s - loss: 0.0160 - acc: 0.99 - ETA: 45s - loss: 0.0155 - acc: 0.99 - ETA: 45s - loss: 0.0218 - acc: 0.99 - ETA: 45s - loss: 0.0236 - acc: 0.99 - ETA: 45s - loss: 0.0227 - acc: 0.99 - ETA: 44s - loss: 0.0218 - acc: 0.99 - ETA: 44s - loss: 0.0223 - acc: 0.99 - ETA: 

In [25]:
model.save_weights('results/weights_densenet_120.h5')

In [13]:
# 50 epochs
model.load_weights('results/weights_densenet_40.h5')

In [26]:
predictions=model.predict(test)

In [27]:
y_pred=np.argmax(predictions,axis=1) 

In [28]:
sub = pd.read_csv('sample_submission.csv')
sub['diagnosis'] = y_pred
#sub.to_csv('submission.csv', index=False)

In [29]:
sub.head()

,id_code,diagnosis
0,0005cfc8afb6,1
1,003f0afdcd15,2
2,006efc72b638,2
3,00836aaacf06,2
4,009245722fa4,4
